# Python WAT

por Flávio Juvenal  
Consultor em Vinta Software  
[vinta.com.br](http://www.vinta.com.br)

## Funções de primeira classe
você sabe o que ocorre quando esquece os parênteses ao chamar funções?

In [ ]:
# WAT 😧
class Programmer:
    def __init__(self, prog_lang):
        self.prog_lang = prog_lang
    
    def is_happy(self):
        return self.prog_lang == 'Python'
    
p = Programmer(prog_lang='Ruby')
p.is_happy  # SyntaxError?

In [ ]:
# Hmm 🤔
p.is_happy()

Como funções (e métodos) em Python são first-class values, o código não fica inválido quando esquecemos os parênteses para chamar funções. Ao invés de chamá-las, temos uma referência a própria função que pode ser usada para passá-la como argumento para outras funções.

In [ ]:
# Nice 🙂
class Programmer:
    def __init__(self, prog_lang):
        self.prog_lang = prog_lang
    
    @property
    def is_happy(self):
        return self.prog_lang == 'Python'

p = Programmer(prog_lang='Python')
p.is_happy

## Sobrescrevendo a Biblioteca Padrão
você sabe o que acontece quando você cria uma variável com um nome igual a um built-in?

In [ ]:
# Hmm 🤔
sum([1,2,3])

In [ ]:
# WAT 😧
sum = 1 + 2  # SyntaxError?
sum([1,2,3])

Use algum editor com syntax-highlighting para saber quando está sobrescrevendo um built-in! Quando precisar dar a uma variável um nome built-in, coloque underline no fim, como em `sum_`.

## Identidade
você sabe quando um objeto em Python é (`is`) outro?

In [ ]:
# Hmm 🤔
a = 256
b = 256
a is b

In [ ]:
# WAT 😧
a = 257
b = 257
a is b

Python tem uma cache de inteiros de [-5, 256], o que faz com que `a` e `b` sejam referências para o mesmo objeto. Como `int`s em Python são imutáveis, essa cache não introduz nenhum bug.

Note que isto é um detalhe da implementação CPython e pode ser diferente em outras implementações. [1]

In [ ]:
# Hmm 🤔
!cat int_example.py

In [ ]:
# WAT 😧
!python int_example.py

Python faz otimizações com as literais que ocorrem em um mesmo arquivo. Se quisermos comparar valor, devemos sempre usar `==` [2]

In [ ]:
# Hmm 🤔
me = {'name': 'Fulano', 'age': 30}
people = [me] * 3
people

In [ ]:
# WAT 😧
people[0]['name'] = 'Sicrano'
people

`[me] * 3` é equivalente a `[me, me, me]`, ou seja, 3 referências para o mesmo objeto.

In [ ]:
# Hmm 🤔
line = [' '] * 3
line

In [ ]:
# Hmm 🤔
game_table = [line] * 3
game_table

In [ ]:
def print_game_table():
    for l in game_table:
        print(l)
print_game_table()

In [ ]:
# WAT 😧
game_table[0][0] = 'X'
print_game_table()

In [ ]:
# Ohh 😮
game_table[0] is game_table[1] and \
game_table[1] is game_table[2]

In [ ]:
# Ohh 😮
id(game_table[0]) == id(game_table[1]) == id(game_table[2])

`id` representa a identidade do objeto. Se duas variáveis tem o mesmo `id`, elas representam o mesmo objeto.  
Em CPython, `id` é o endereço do objeto na memória. [3]


In [ ]:
# WAT 😧
game_table[2][0] = 'O'
print_game_table()

In [ ]:
# Nice 🙂
game_table = [[' ' for j in range(3)] for i in range(3)]
print_game_table()

In [ ]:
# Nice 🙂
game_table[2][0] = 'O'
print_game_table()

In [ ]:
# Hmm 🤔
game_table = [[' '] * 3 for i in range(3)]
print_game_table()

In [ ]:
# WAT 😧
game_table = [[{}] * 3 for i in range(3)]
game_table[0][0]['name'] = 'Fulano'
print_game_table()

Resumindo:
- cuidado ao atribuir variáveis, atribuição != cópia
- cuidado ao multiplicar listas, multiplicação de listas != cópia
- use `id` para ver qual objeto uma variável representa
- use `is` para ver se duas variáveis representam o mesmo objeto
- para tabelas, faça dois `for`s

## Atribuição em tuplas
tuplas são imutáveis, mas tem alguma maneira de mudar seus elementos?

In [ ]:
# Hmm 🤔
a = ([42],)
a[0] += [43]

In [ ]:
# WAT 😧
a

`+=` é operador de adição in-place. Ele faz duas coisas:
- ele chama `obj.__iadd__(rhs)`, que **pode** modificar o objeto caso ele seja mutável.
- ele atribui o retorno de `obj.__iadd__(rhs)` para a variável.

Apenas esta segunda ação falha no `a[0] += [43]` quando `a` é uma tupla. A primeira é feita! E, para listas, a primeira ação modifica a lista. [8] [9]


In [ ]:
# WAT 😧
l = [42]
l + [43]
l

Não confunda! O `+` não tem comportamento in-place. Ele retorna um novo objeto (pelo menos para os tipos built-in do Python).

## Variáveis de classe
você está usando variáveis de classe corretamente?

In [ ]:
# Hmm 🤔
class Dog:
    tricks = []

    def __init__(self, name):
        self.name = name
    
    def add_trick(self, trick):
        self.tricks.append(trick)

    def print_tricks(self):
        print(self.name, ' tricks:')
        for trick in self.tricks:
            print(trick)

In [ ]:
# Hmm 🤔
teddy = Dog("Teddy")
teddy.add_trick("roll over")
teddy.print_tricks()

In [ ]:
# WAT 😧
kika = Dog("Kika")
kika.add_trick("catch ball")
kika.print_tricks()

In [ ]:
# WAT 😧
teddy.print_tricks()

In [ ]:
# Nice 🙂
class Dog:
    def __init__(self, name):
        self.tricks = []
        self.name = name

    # ...

Cuidado com a diferença entre variáveis de classe e de instância. [10]

Exemplo de bug real: [11]

In [ ]:
# Hmm 🤔
class A:
    x = 1

class B(A):
    pass

class C(A):
    pass

print(A.x, B.x, C.x)

In [ ]:
# Hmm 🤔
B.x = 2
print(A.x, B.x, C.x)

In [ ]:
# WAT 😧
A.x = 3
print(A.x, B.x, C.x)

Quando um atributo ou método não é encontrado em uma classe, ele é procurado nas suas classes mães, seguindo o MRO (Method Resolution Order)

In [ ]:
# WAT 😧
class A:
    x = 1

class B(A):
    pass

a_instance = A()
a_instance.x = 2
print("A.x", A.x)
print("B.x", B.x)

Se uma variável é atribuída através da instância, ela é uma variável de instância. Se é atribuída através da classe, ela é uma variável de classe. [12] [13]

## Escopo
você entende as regras de escopo de Python?

In [ ]:
# Hmm 🤔
x = 10
def next_x():
    return x + 1
next_x()

In [ ]:
# WAT 😧
x = 10
def increment():
    x += 1
    return x
increment()

Quando você faz uma atribuição para uma variável em um escopo, essa variável é automaticamente considerada como local nesse escopo e esconde qualquer variável com o mesmo nome no escopo externo. [4]


In [ ]:
# WAT 😧
def counter():
    x = 10
    def increment_aux():
        x += 1
        return x
    return increment_aux()
counter()

A solução para não criar uma nova variável local e fazer uma atribuição é usar as keywords `global` ou `nonlocal`

In [ ]:
# Nice 🙂
x = 10
def increment():
    global x
    x += 1
    return x
increment()

In [ ]:
# Nice 🙂
def counter():
    x = 10
    def increment_aux():
        nonlocal x
        x += 1
        return x
    return increment_aux()
counter()

LEGB rule:
- **atribuições** em Local, Enclosing, Global, Built-in definem variáveis no seu escopo
- se uma variável for usada, ela será procurada na **ordem** LEGB

In [ ]:
# Nice 🙂
def heads_or_tails(is_head):
    if is_head:
        coin = 'heads'
    else:
        coin = 'tails'
    print(coin)
heads_or_tails(is_head=False)

In [ ]:
# Nice 🙂
print(type)  # built-in
type = 'global'
def generate_fn():
    type = 'enclosing'
    def fn():
        type = 'local'
        return type
    return fn
generate_fn()()

In [ ]:
# Ohh 😮
import dis
a = 1
def my_a():
    return a
dis.dis(my_a)

In [ ]:
# Ohh 😮
a = 1
def my_a():
    a += 1
    return a

dis.dis(my_a)

LOAD_GLOBAL: Loads the global onto the stack.  
LOAD_FAST: Pushes a reference to the local onto the stack. [5]


## Evaluation de Default Arguments
você sabe quando default arguments são avaliados?

In [ ]:
# Hmm 🤔
def add_poddle(dogs=[]):
    dogs.append("poddle")
    return dogs

In [ ]:
# Hmm 🤔
add_poddle(['bulldog'])

In [ ]:
# WAT 😧
print(add_poddle())
print(add_poddle())

Default arguments são avaliados no momento da definição da função.
Se eles forem mutáveis (como uma lista), vão compartilhar estado durante diferentes chamadas,
o que normalmente é indesejado. [6]


In [ ]:
# Hmm 🤔
add_poddle.__defaults__

In [ ]:
# Ohh 😮
add_poddle.__defaults__[0].append('ohh')
add_poddle()

## Late Binding em Closures
closures acessam variáveis como você espera?

In [ ]:
# WAT 😧
def create_multipliers():
    return [lambda x : i * x for i in range(5)]

for multiplier in create_multipliers():
    print(multiplier(2))

As closures em Python são late-binding. Ou seja, os valores de variáveis são acessados
só no momento em que a closure é chamada.

Isso não é exclusivo para `lambda`, o mesmo ocorre com `def`. [7]


In [ ]:
# Nice 🙂
def create_multipliers():
    return [lambda x, i=i : i * x for i in range(5)]

for multiplier in create_multipliers():
    print(multiplier(2))

A solução é usar o WTF anterior, já que Default Arguments são avaliados no momento de definição!

## Dúvidas?
Confuso? Todos Estamos!
😧😧😧

Fontes:  
[1] https://docs.python.org/3/c-api/long.html#c.PyLong_FromLong  
[2] http://stackoverflow.com/a/15172182/145349  
[3] https://docs.python.org/3/library/functions.html#id  
[4] https://docs.python.org/3/faq/programming.html#why-am-i-getting-an-unboundlocalerror-when-the-variable-has-a-value  
[5] https://docs.python.org/2/library/dis.html#opcode-LOAD_FAST  
[6] https://docs.python.org/3/faq/programming.html#why-are-default-values-shared-between-objects  
[7] https://docs.python.org/3/faq/programming.html#why-do-lambdas-defined-in-a-loop-with-different-values-all-return-the-same-result  
[8] http://stackoverflow.com/a/21361412/145349  
[9] https://docs.python.org/3/faq/programming.html#why-does-a-tuple-i-item-raise-an-exception-when-the-addition-works  
[10] https://docs.python.org/3/tutorial/classes.html#class-and-instance-variables  
[11] https://github.com/allisson/django-pagseguro2/pull/6  
[12] https://www.toptal.com/python/top-10-mistakes-that-python-programmers-make#common-mistake-2-using-class-variables-incorrectly  
[13] https://www.toptal.com/python/python-class-attributes-an-overly-thorough-guide#handling-assignment

## Obrigado!
[@vintasoftware](https://twitter.com/vintasoftware/)  
[@flaviojuvenal](https://twitter.com/flaviojuvenal/)